# Home project task package 2

In this home project, each group will be assigned a dataset of one of the 4 properties: band gap, formation energy, Poisson ratio and magnetization. In the following blocks, you will go through a demostration first to gain a feeling of the whole workflow (Task 1-6). Then you can import the datasets to the machine learning according to your learning objectives (Task 7&8).

# Data import and inspection

For each datasets, you can check the data format by using dataframe.describe() function. The last coloumn is the target property and the rest columns are the corresponding feartures of the compounds. The features are extracted rom crystal structures of inorganic compunds using magpie (Material-Agnostic Platform for Informatics and Exploration, website: https://wolverton.bitbucket.io/index.html).

In [ ]:
import pandas as pd

data = pd.read_csv('Tc.csv')  ### here to select the input data file
print('Dataset shape:', data.shape)
data.describe()

In [ ]:
from sklearn.utils import shuffle

features = list(data)[2:-1]
properties = list(data)[-1]
data = shuffle(data, random_state=0)
X = data[features]
y = data[properties]
print('number of features:', len(features))

In machine learning (ML), besides collecting the efficient data, most of time we will also meet some problems that the features used to analyze are either redundent or inter-correlated, and we want a way to create a model that only includes the most important features. To solve such problems, we then need some methods to select the efficient features as well. There are three benefits for doing so. First, we make our model more simple to interpret. Second, we can reduce the variance of the model, and therefore the overfitting. Finally, we can reduce the computational cost (and time) of training a model. The process of identifying only the most relevant features is called “feature selection.”

Among most of ML model and statistical methods, random forests (RF) are often used for feature selection in a data science workflow. The reason is because the tree-based strategies used by random forests naturally ranks by how well they improve the purity of the node. This mean decrease in impurity over all trees (called gini impurity). Nodes with the greatest decrease in impurity happen at the start of the trees, while notes with the least decrease in impurity occur at the end of trees. Thus, by pruning trees below a particular node, we can create a subset of the most important features. In this tutorial, we will mainly take the advantage of RF to do the feature selection, but meanwhile some other type of feature selection method will also used for pre-screening.

In the first round, we will do a rough regression model evaluation with the help of corss validation (CV) to test the performance of your selected model on the datasets using all features.
### Task 1: A initial evaluation of your selected model on datasets with all features. Here your task is try to fill ML model code and cross validation code to get familiar with the selected ML algorithm, you can check these codes on sklearn web page. 

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

### Task1: here try to fill ML model code and cross validation code, you can check these code on sklearn web page. 

# reg = RandomForestRegressor()  #fill here 
# scores = cross_val_score()  # fill here

### result
scores.mean()

There are several methods of feature selection in practice. For the baisc feature screening, here we choose a simple api function given in Sklearn's feature_selection module called VarianceThreshold, whoose function is to remove features with low variance, which means the variation of these features are not huge according to the training data. VarianceThreshold is a simple basic method of feature selection that removes all features whose variance does not meet some threshold. By default, it removes all features that have a variance of 0, i.e. all features that take the same value.

In [ ]:
from sklearn.feature_selection import VarianceThreshold
# Get features that return at least 95% of the Bernoulli random variables
sel =  VarianceThreshold(threshold=(0.95*(1-0.95)))
# You can see which features have been retained
X_sel = sel.fit_transform(X)
# Output results
feature_index = sel.get_support()
print('number of features after remove redundant ones:',sum(feature_index))

## list of final features
final_features=[]

for i in range(len(features)):
    if feature_index[i]==True:
        final_features.append(features[i])

One of the key steps in Machine learning models is to normalize the feature data, the function of normalisation are: 

(a) Normalisation speeds up gradient descent to find the optimal solution.
Normalisation is often necessary if the machine learning model uses gradient descent to find the optimal solution, otherwise it will be difficult or even impossible to converge.

(b) Normalisation has the potential to improve accuracy.

Some classifiers need to calculate distances (Euclidean distances) between samples, e.g. KNN. If a feature has a very large range of values, then the distance calculation depends mainly on this feature, thus contradicting the actual situation (e.g. in this case the actual situation is that features with a small range of values are more important).

But of course not all of the ML models need normalisation. For example, probabilistic models (tree models) do not require normalisation because they are not concerned with the values of the variables, but with the distribution of the variables and the conditional probabilities between them, e.g. decision trees, RF. Whereas optimization problems like Adaboost, SVM, LR, Knn, KMeans, they require normalisation. In case the ML model may need normalisation, here we normalized the data first.

### Task 2.1: Check sklearn website and find out how to do normalisation, please return a variable with name X_sc. 

In [ ]:
from sklearn.preprocessing import StandardScaler

### Task 2.1: Check sklearn website and find out how to do normalisation, 
# please return a variable with name X_sc.
scaler = StandardScaler()
# ...

### result 
print('X shape:', X_sc.shape)

Then we will using the correlation heat map to analyze the inter-correlation between each feature, and we will only select the ones whcich are most representative. The Spearman rank-order correlation coefficient is a nonparametric measure of the monotonicity of the relationship between two datasets, and it does not assume that both datasets are normally distributed. Thus here we choose it as the function for calculating correlation matrix.

In [ ]:
from scipy.stats import spearmanr
corr = spearmanr(X_sc).correlation
print(corr.shape)

Perform Ward’s linkage on a condensed distance matrix, get the hierarchical clustering (corr_linkage).

### Task 2.2: please use the hierarchical clustering (corr_linkage) and final_features you get in task1 to make a dendrogarm

In [ ]:
from scipy.cluster import hierarchy

corr_linkage = hierarchy.ward(corr)

import matplotlib.pyplot as plt
### Task 2.2: please use the hierarchical clustering (corr_linkage) 
# and final_features you get in task1 to make a dendrogarm named dendro.
#
fig, ax = plt.subplots(1, 1, figsize=(10,10))
# dendro = hierarchy.dendrogram() # fill here
plt.show()


In [ ]:
print(dendro['leaves'])
print(dendro['ivl'])

### Task 2.3: Based on this hierarchical clustering order, please visualize the correlations and plot them out the heat map with colorbar using dendro['leaves'] 

In [ ]:
import numpy as np
### Task 2.3: Based on this hierarchical clustering order, 
# please visualize the correlations 
# and plot out the heat map with colorbar using dendro['leaves'] 


We flat the corr_linkage matrix by using fcluster function. Now please think about what is this 't=2' means in the following 'cluster_ids' code. 

### Task 3: Please try to explain: 1. what are the meanings of 't=2' and 'distance' in the following 'cluster_ids' code; 2. The relationship between corr_likeage matrix and our final cluster_id_to_feature_ids;  3. Finally in selected_features, what are these selected features?

In [ ]:
cluster_ids = hierarchy.fcluster(corr_linkage, t=2, criterion='distance')
from collections import defaultdict
cluster_id_to_feature_ids = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids):
    cluster_id_to_feature_ids[cluster_id].append(idx)
print(cluster_id_to_feature_ids)
print(cluster_id_to_feature_ids.values())
selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]
print(selected_features)
print('number of features after correlation reduction:',len(selected_features))

Now with all these informative features, let's start training.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=10, random_state=0)
X_train=X_train[:,selected_features]
X_test=X_test[:,selected_features]

Here as a example, we will show a pipeline random forest code combining SelectKBest and RandomForestRegressor. The reason to use RF here is because later in the importance analysis part we need to use RF. However, in the final step, you can also use the same procedure to select your best hyperparameter combinations for your selected ML model.

SelectKBest will select features according to the k highest scores, the default value is 10 in sklearn. And in SelectKBest, we have to use f_regression to make it workable for regression problems. Then we will set some hyperparameter sets, by combining the pipeline model and cross validation methods, we can do a grid search to choose the best combination of these hyperparameters.

### Task 4: Try to set and tune these hyperparameters by yourself, and select the best combinations

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

skb = SelectKBest(score_func=f_regression)
est_rf = RandomForestRegressor(random_state=0)
pipe_rf = Pipeline([('SKB', skb), ('forest', est_rf)])

### Complete the lines below
# param_grid_rf = {
#     'forest__n_estimators':[],
#     'forest__max_depth':[],
#     'forest__max_features':[],
#     'forest__min_samples_leaf':[]
# } 

In [ ]:
from sklearn.model_selection import GridSearchCV

### Complete the lines below
# gcv_rf = GridSearchCV() #fill here
cv_rf.fit(X_train, y_train)
print('\nRandomForest:', gcv_rf.best_score_)
print(gcv_rf.best_params_)

Based on your previous hyperparameter sets, in this section we will try 1000 random initial state to find the best performance of the model. Next we will record this random intial state and use it as the data spliting parameter for train and validation sets.

In [ ]:
#here please fill the arguments with the best parameters you found in last step
#est_rf = RandomForestRegressor(random_state=,max_features=' ',n_estimators= , max_depth= , min_samples_leaf= ) 

best_results=-10
best_state=-1

for i in range (1000):
    X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=20, random_state=i)
    X_train=X_train[:,selected_features]
    X_test=X_test[:,selected_features]
    est_rf.fit(X_train,y_train)
    result=est_rf.score(X_test, y_test)
    if result>best_results:
        best_results=result
        best_state=i

print(best_results,best_state)

X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=20, random_state=best_state)
X_train=X_train[:,selected_features]
X_test=X_test[:,selected_features]

Now use the permutation_importance function to evaluate the importance of each feature. The permutation importance of one feature is calculated as follows. First, a baseline metric, defined by scoring (if is None, then a default scoring of estimator will be used, please refer to the permutation_importance web page in sklearn), is evaluated on a (potentially different) dataset defined by the X. Next, a feature column from the validation set is permuted and the metric is evaluated again. The permutation importance is defined to be the difference between the baseline metric and metric from permutating the feature column.

### Task 5: check and learn how to use the permutation_importance function and generate a sorted mean importance results with name 'perm_sorted_idx'.

In [ ]:
from sklearn.inspection import permutation_importance

### Task 5

# result = permutation_importance() # please fill here
perm_sorted_idx = result.importances_mean.argsort()


With the generated sorted mean importances, let's ploted it out for visualization.

In [ ]:
tree_importance_sorted_idx = np.argsort(est_rf.feature_importances_)
tree_indices = np.arange(0, len(est_rf.feature_importances_)) + 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
ax1.barh(tree_indices, est_rf.feature_importances_[tree_importance_sorted_idx], height=0.7)

ylabels=[]
for i in range(len(tree_importance_sorted_idx)):
    ylabels.append(final_features[i])


ax1.set_yticklabels(ylabels)
ax1.set_yticks(tree_indices)
ax1.set_ylim((0, len(est_rf.feature_importances_)))

labels=[]
for i in range(len(perm_sorted_idx)):
    labels.append(final_features[i])   
    
#print(labels,tree_importance_sorted_idx,est_rf.feature_importances_)
ax2.boxplot(result.importances[perm_sorted_idx].T[-10:], vert=False, labels=labels)
fig.tight_layout()
plt.show()

Now with these plots, you will have a straight-forward impression on how will each feature affects the target properties. You can now select out some most important features by yourself and using them as the input for next ML model training, or you can automatically select the features with the help of 'SelectFromModel' function in sklearn. Below is a simple example for illustration.

In [ ]:
from sklearn.feature_selection import SelectFromModel

# Create a random forest regressor
ref = RandomForestRegressor(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
ref.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature, importance in zip(final_features, ref.feature_importances_):
    print(feature, importance)
    
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.15
sfm = SelectFromModel(ref, threshold=0.05)

# Train the selector
sfm.fit(X_train, y_train)

# Print the names of the most important features
for feature_list_index in sfm.get_support(indices=True):
    print(final_features[feature_list_index])
    
# Transform the data to create a new dataset containing only the most important features
# Note: We have to apply the transform to both the training X and test X data.
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)

final_index = sfm.get_support(indices=True)
print(final_index)
print([final_features[i] for i in final_index])

### Task 6: Now try to use the obtained trained model to predict the properties of test sets to evaulate the performance of the obtained RF model or the customized model by yourself, and make a comparation plot to visualize the differences between your predict and target result.

In [ ]:
### Task 6

### Task 7: Tasks 1-6 have provided detailed guidance on the use of random forest algorithm,  please try to apply another type of ML algorithms (e.g. SVR, GP, NN ...) on the training of this dataset.

### Task 8: With what you have learned in these tasks, please do ML on your own dataset as shown in this code.      

Hint for Task 7 & 8: since the dataset's size is much larger than that of test dataset, you need to consider the size of train set and the parameters to balance the training speed and accuracy. To avoid the contingency of relatively small training sets, you can use multiple training sets and average their results(batch training is also recommended). The best performing training set is often the most representative data, and you can save it.

### Task 9: Uncertainty estimation

The previous 8 tasks, contains common methods for improving our machine learning models. In the process, we also found that the size and quality of the training set has a significant impact on the model performance. However, in practice, we often cannot specify a certain proportion of the whole material discovery space or certain materials as a training set as we wish. If you're an experimentalist or simulation calculator, you're probably used to this kind of scenarios: days and weeks of effort to obtain the target properties of only one material. 

This has led to many machine learning models that aim to make predictions about material systems struggling. So in this case do we have the opportunity to find the interested candicates in by optimizing our sampling? **Uncertainty estimation might be the key to this question.**   

Uncertainty estimation allows researchers to understand the reliability of these conclusions and provides a measure of how representative the small sample is of the entire materials space. In addition, by quantifying the uncertainty, researchers can determine the range of values or variability that might exist across the materials space. This understanding is crucial for extrapolating findings and making predictions for materials that were not part of the original sample.

To obtain uncertainty, **bootstrapping method** is one way, consider that we only have a small part of data randomly picked from the large dataset, first choose one machine learnng algrithm on this train set (e.g random forest) and train a model. Then we resample the data and train a new model. By doing such bootsrapped sampling for some times, each sample in the trainset is in generally equally sampled and the models obtain variety. For each candicate in the whole large dataset, there is a bunch of predictions corresponding to each bootstrapping model. The stadard deviation of these predictions provide us with the uncertainty. 

In this task please realize the uncertainty estimation of the prediction. You can use either the [bootstraping method](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) described above (It's both time- and resource- consuming due to the resamplings and retrainings), or any other method you have in mind. **(you will get an 0.3 bonus for your effort to save time and calculation resource!!!)**


In [ ]:
### Task 9

### Task 10: Active learning, Exporation or Optimization?

From the previous Task 9 we have an idea of how to estimate the uncertainty. In our case, the uncertainty can help us both in **exploration of materials space and identifying promising candidates.**

With a small sample size relative to the materials space, it is essential to develop **efficient and effective sampling strategies**. Uncertainty estimation can guide the design of these strategies by providing insights into the variability and uncertainty associated with different sampling approaches. Researchers can use the estimated uncertainties to determine the optimal allocation of resources and prioritize the sampling of materials that are likely to provide the most valuable information.

Moreover, in the field of materials discovery, the identification of promising materials for specific applications is a key objective. Uncertainty estimation plays a vital role in assessing the reliability of performance predictions and property assessments for these candidate materials. By quantifying the uncertainty, researchers can rely on the level of confidence associated with the predictions, allowing them to prioritize and concentrate their efforts on **the most promising candidates for further exploration.** 

Then for Task 10, establishing an active learning loop is required. **YOU NEED TO CLARIFY YOUR GOAL**. Use your active learning loop to help you train a high performance ML model with a small percentage of the whole data? Or you wanna find those extremelly high performance materials in limited iterations?
 
Some links might be helpful:

https://towardsdatascience.com/active-learning-in-machine-learning-525e61be16e5

https://distill.pub/2020/bayesian-optimization/

In [ ]:
### Task 10

### Important notes:

1. For Task 7&8, please provide us with your final trained models(for sklearn, save model using joblib), we will generate **5 unknown test datasets** for each properties, your final models will be evaluated on these 5 test sets. **Please note** that this is just a validation for us and don't worry if your model is not so perfect. 

2. For Task 9-10, please wrap up your **all** results, plots and discussions to one or several Jupyter notebooks. **Your home project will be graded based on the report.** 


3. **Updated grading criteria**:
    
    The group finishing Task 1-8 are guaranteed to get grade within 2.7-4.0. \
    The group finishing up tp Task 9 get at least 2.3. \
    The group finishing all the tasks get at least 1.3.
    
Please let your supervisor Xiankang Tang know if you have any question. (xtang@tmm.tu-darmstadt.de)

Good Luck!（‐＾▽＾‐）